<a href="https://colab.research.google.com/github/vlassner/DSML_4220_Deep_Learning/blob/main/Lab9_finetuning_gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 9: Finetuning GPT-2 with LoRA

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sgeinitz/DSML4220/blob/main/lab9_finetuning_gpt2.ipynb)

[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/sgeinitz/DSML4220/blob/main/lab9_finetuning_gpt2.ipynb)

In this lab we will use GPT-2 for the task of text generation. We'll first quickly compare Greedy Search and (Diverse) Beam Search with GPT-2. Then we'll finetune GPT-2 to generate text that is more explicitly infused with knowledge of Hemingway's book, "_The Sun also Rises_", and can generate text in the style of the book.


### Lab 9 Assignment/Task
There are three questions in this lab. As an added bonus, try downloading your own book from Project Gutenberg to finetune GPT-2 to generate text following your chosen book/author (see [this script](https://github.com/sgeinitz/DSML4220/blob/main/convert_book2csv.py)) for help to convert it to a .csv file of sentences).

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


import numpy as np
import pandas as pd

from transformers import GPT2Tokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from torch.utils.data import Dataset, random_split
from peft import LoraModel, LoraConfig

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

In [ ]:
inputs = tokenizer(["Today is"], return_tensors="pt")
inputs

Let's generate some text from the model using regular Greedy Search (here is the [HuggingFace example documenting this](https://huggingface.co/docs/transformers/en/main_classes/text_generation#transformers.GenerationMixin.compute_transition_scores.example)).

In [ ]:
# Example 1: Print the scores for each token generated with Greedy Search
#tokenizer.pad_token_id = tokenizer.eso_token_id
outputs = model.generate(**inputs, max_new_tokens=10, return_dict_in_generate=True, output_scores=True)
transition_scores = model.compute_transition_scores(
    outputs.sequences, outputs.scores, normalize_logits=True
)
# input_length is the length of the input prompt for decoder-only models, like the GPT family, and 1 for
# encoder-decoder models, like BART or T5.
input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
generated_tokens = outputs.sequences[:, input_length:]
for tok, score in zip(generated_tokens[0], transition_scores[0]):
    # | token | token string | log probability | probability
    print(f"| {tok:5d} | {tokenizer.decode(tok):8s} | {score.numpy():.3f} | {np.exp(score.numpy()):.2%}")

In [ ]:
outputs['sequences']

Let's now use Beam Search (again using [this example from HF](https://huggingface.co/docs/transformers/en/main_classes/text_generation#transformers.GenerationMixin.compute_transition_scores.example)).

In [ ]:
inputs = tokenizer(["Today is"], return_tensors="pt")

# Approach 2: Beam Search
outputs = model.generate(
    **inputs,
    max_new_tokens=10,
    num_beams=6,
    num_beam_groups=3,
    diversity_penalty=5.0,
    num_return_sequences=6,
    return_dict_in_generate=True,
    output_scores=True,
)
transition_scores = model.compute_transition_scores(
    outputs.sequences, outputs.scores, outputs.beam_indices, normalize_logits=False
)

In [ ]:
outputs['sequences']

In [ ]:
for s, seq in enumerate(outputs['sequences']):
  print(f"seq {s}: {tokenizer.decode(seq)}")

---

### Q1: Does the Beam Search above use Diverse Beam Search? If not, change it to use Diverse Beam Search and describe how the output differs.  

(Hint: Look a few cells down at the next use of Beam Search, there are two parameters you will need to add, `num_beam_groups`, and `diversity_penalty`)

The outputs differ vastly when num_beams_groups and diversity_penatly are added. Without them, the sentences had very few differences-the last few words. However, those added variables added 3 separate scenarios of possible words after "Today is".
---

In [ ]:
prompt = ["Cohn confronted the bullfighter and "]
inputs = tokenizer(prompt, return_tensors="pt")

max_new_toks = 15
# Example 1: Print the scores for each token generated with Greedy Search
#outputs = model.generate(**inputs, max_new_tokens=max_new_toks, return_dict_in_generate=True, output_scores=True, do_sample=True, temperature=1)
outputs = model.generate(**inputs, max_new_tokens=max_new_toks, return_dict_in_generate=True, output_scores=True, do_sample=False)
transition_scores = model.compute_transition_scores(
    outputs.sequences, outputs.scores, normalize_logits=True
)
# input_length is the length of the input prompt for decoder-only models, like the GPT family, and 1 for
# encoder-decoder models, like BART or T5.
input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
generated_tokens = outputs.sequences[:, input_length:]
for tok, score in zip(generated_tokens[0], transition_scores[0]):
    # | token | token string | log probability | probability
    print(f"| {tok:5d} | {tokenizer.decode(tok):8s} | {score.numpy():.3f} | {np.exp(score.numpy()):.2%}")

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt")

# Approach 2: Reconstruct the sequence scores from Beam Search
outputs = model.generate(
    **inputs,
    max_new_tokens=max_new_toks,
    num_beams=6,
    num_beam_groups=3,
    diversity_penalty=5.0,
    num_return_sequences=6,
    return_dict_in_generate=True,
    output_scores=True,
    temperature=1.0,
    #do_sample=True
)
transition_scores = model.compute_transition_scores(
    outputs.sequences, outputs.scores, outputs.beam_indices, normalize_logits=False
)
# If you sum the generated tokens' scores and apply the length penalty, you'll get the sequence scores.
# Tip 1: recomputing the scores is only guaranteed to match with `normalize_logits=False`. Depending on the
# use case, you might want to recompute it with `normalize_logits=True`.
# Tip 2: the output length does NOT include the input length
output_length = np.sum(transition_scores.numpy() < 0, axis=1)
length_penalty = model.generation_config.length_penalty
reconstructed_scores = transition_scores.sum(axis=1) / (output_length**length_penalty)

print(np.allclose(outputs.sequences_scores, reconstructed_scores))

for s, seq in enumerate(outputs['sequences']):
  print(f"seq {s}: {tokenizer.decode(seq)}")

Let's now load the raw text from Hemingway's book, _"The Sun also Rises"_.

In [ ]:
heming = pd.read_csv("https://raw.githubusercontent.com/sgeinitz/DSML4220/main/data/sunalsorises.csv")
heming.head()

In [ ]:
sentences = heming['sentence']
sentences.head()

In [ ]:
print(f"        sentence: '{sentences[0]}' \n is tokenized as: {tokenizer.encode(sentences[0])}")

In [ ]:
max_length = max([len(tokenizer.encode(sentence)) for sentence in sentences])
max_length

In [ ]:
class HemingwayDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            encodings_dict = tokenizer('<|startoftext|>' + txt + '<|endoftext|>', truncation=True,
                                       max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self): # overload the len() Python built-in function
        return len(self.input_ids)

    def __getitem__(self, idx): # overload the [] operator
        return self.input_ids[idx], self.attn_masks[idx]

tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
dataset = HemingwayDataset(sentences, tokenizer, max_length=max_length)
train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

In [ ]:
train_dataset[0]

Notice that above we set the `pad_token_id` to be the same as the `eos_token_id` (i.e. end-of-stream token id). So all of those `50256` entries above are being used as end-of-stream, or end-of-sequence tokens (except the first one, which is denoting the end of the sequence).  

In [ ]:
tokenizer.decode([50256])

In [ ]:
batch_size = 4
n_epochs = 2
training_args = TrainingArguments(output_dir='~/hemingway_generation', num_train_epochs=n_epochs, logging_steps=100, save_steps=500, do_eval=True,
                                  eval_steps=20, per_device_train_batch_size=batch_size, per_device_eval_batch_size=batch_size, save_safetensors=False,
                                  warmup_steps=10, weight_decay=0.05, logging_dir='~/hemingway_generation/logs', report_to='none')

Let's load GPT-2 and then  take a rough glance at the architecture of GPT (w/ ~130M parameters) by printing the model.

In [ ]:
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")
print(model)

In [ ]:
def count_trainable_parameters(mod):
    model_parameters = filter(lambda p: p.requires_grad, mod.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    return params

gpt2_params = count_trainable_parameters(model)
print(f"GPT-2 trainable parameters: {gpt2_params}")

In [ ]:
trainer = Trainer(model=model,  args=training_args, train_dataset=train_dataset, eval_dataset=val_dataset,
                  data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                              'attention_mask': torch.stack([f[1] for f in data]),
                                              'labels': torch.stack([f[0] for f in data])})
# on Colab this will take 6+hrs w/ cpu or <10min w/ T4 GPU per epoch
trainer.train()

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Use Diverse Beam Search
outputs = model.generate(
    **inputs,
    max_new_tokens=max_new_toks,
    num_beams=6,
    num_beam_groups=3,
    diversity_penalty=5.0,
    num_return_sequences=5,
    return_dict_in_generate=True,
    output_scores=True,
    temperature=2.0,
    #do_sample=True
)
transition_scores = model.compute_transition_scores(
    outputs.sequences, outputs.scores, outputs.beam_indices, normalize_logits=False
)

output_length = np.sum(transition_scores.cpu().numpy() < 0, axis=1)
length_penalty = model.generation_config.length_penalty
reconstructed_scores = transition_scores.cpu().sum(axis=1) / (output_length**length_penalty)

for s, seq in enumerate(outputs['sequences']):
  gen_text = tokenizer.decode(seq)
  # remove everything from '<|endoftext|>' on at the end of gen_text
  gen_text = gen_text[:gen_text.find('<|endoftext|>')]
  print(f"seq {s}: {gen_text}")

Next, let's use LoRA to fine tune the model. We'll load the model again to ensure that the earlier finetuning is not included.

In [ ]:
# load the model again so that we can use LoRA
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")
print(model)

In [ ]:
target_modules = ["q_proj", "k_proj", "v_proj", "out_proj", "fc_in", "fc_out", "wte", "c_fc", "c_proj"]

lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    inference_mode=False,
    r=16,
    lora_alpha=32,
    target_modules=target_modules,
    lora_dropout=0.01,
    fan_in_fan_out=True
)

lora_model = LoraModel(model, lora_config, "default")
lora_model.model.tie_weights()

In [ ]:
print(lora_model)

In [ ]:
trainer = Trainer(model=lora_model,  args=training_args, train_dataset=train_dataset, eval_dataset=val_dataset,
                  data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                              'attention_mask': torch.stack([f[1] for f in data]),
                                              'labels': torch.stack([f[0] for f in data])})

In [ ]:
trainer.train()

---

### Q2: How many more `training_samples_per_second` could the LoRA model get through during finetuning than the original GPT-2 model could?

The LoRA was able to get through 2 more training_samples_per_second than the GPT-2 model.

---

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Use Diverse Beam Search
outputs = lora_model.generate(
    **inputs,
    max_new_tokens=max_new_toks,
    num_beams=6,
    num_beam_groups=3,
    diversity_penalty=5.0,
    num_return_sequences=5,
    return_dict_in_generate=True,
    output_scores=True,
    temperature=1.5,
    #do_sample=True
)
transition_scores = lora_model.compute_transition_scores(
    outputs.sequences, outputs.scores, outputs.beam_indices, normalize_logits=False
)

output_length = np.sum(transition_scores.cpu().numpy() < 0, axis=1)
length_penalty = lora_model.generation_config.length_penalty
reconstructed_scores = transition_scores.cpu().sum(axis=1) / (output_length**length_penalty)

for s, seq in enumerate(outputs['sequences']):
  gen_text = tokenizer.decode(seq)
  # remove everything from '<|endoftext|>' to the end from gen_text
  gen_text = gen_text[:gen_text.find('<|endoftext|>')]
  print(f"seq {s}: {gen_text}")

In [ ]:
lora_params = count_trainable_parameters(lora_model)
print(f"LoRA trainable parameters: {lora_params} ({(100*lora_params/gpt2_params):.2f}% of GPT-2's trainable parameters)")

---

### Q3: How many fewer parameters did the LoRA model need to train/tune than the full GPT-2 model did?

(Hint: See output from above cell)

It used 2.08% of GPT-2's trainable parameters.

---